In [88]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.dml import MSO_FILL_TYPE, MSO_THEME_COLOR
from pptx.enum.text import MSO_ANCHOR
from pptx.dml.color import RGBColor

In [101]:
data = {
    "name": "MathUp",
    "brief": "Приложение для запоминания математических формул",
    "bg": {
        "type": "gradient",
        "color1": "a3fff9",
        "color2": "a3b4ff",
        "angle": 45,
    },
    "logo": {"size": 0.7, "path": "logo.png"},
    "font": {"name": "VK Sans Display", "title_size": 50, "regular_size": 38},
}

PRESENTATION_WIDTH = Inches(16)
PRESENTATION_HEIGHT = Inches(9)

In [102]:
def add_custom_logo(presentation, slide):
    border_offset = Inches(0.25) + Inches(data["logo"]["size"])

    slide.shapes.add_picture(
        data["logo"]["path"],
        presentation.slide_width - border_offset,
        presentation.slide_height - border_offset,
        width=Inches(data["logo"]["size"]),
    )

In [103]:
from pptx.dml.color import RGBColor


def apply_text_formatting(
    obj, color: str = "000000", bold: bool = False, is_title: bool = False
):
    obj.text_frame.paragraphs[0].font.name = data["font"]["name"]
    obj.text_frame.paragraphs[0].font.bold = bold
    obj.text_frame.paragraphs[0].font.color.rgb = RGBColor.from_string(color)
    font_size = data["font"]["title_size"] if is_title else data["font"]["regular_size"]
    obj.text_frame.paragraphs[0].font.size = Pt(font_size)

In [105]:
# Create a new presentation object
prs = Presentation()
prs.slide_width = PRESENTATION_WIDTH
prs.slide_height = PRESENTATION_HEIGHT

# Define slide layout (0 is the layout for a title slide)
slide_layout = prs.slide_layouts[0]

# Add a slide with the defined layout
slide = prs.slides.add_slide(slide_layout)

background = slide.background
fill = background.fill


# SLIDE BACKGROUND
if data["bg"]["type"] == "gradient":
    fill.gradient()
    fill.gradient_stops[0].color.rgb = RGBColor.from_string(data["bg"]["color1"])
    fill.gradient_stops[1].color.rgb = RGBColor.from_string(data["bg"]["color2"])
    fill.gradient_angle = data["bg"]["angle"]
elif data["bg"]["type"] == "solid":
    fill.solid()
    fill.fore_color.rgb = RGBColor.from_string(data["bg"]["color"])


title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = data["name"]
title.width = PRESENTATION_WIDTH - Inches(2)
title.left = Inches(1)
title.top = Inches(3.5)

subtitle.text = data["brief"]
subtitle.width = Inches(14)
subtitle.left = Inches(1)
subtitle.top = Inches(3.5) + Inches(1)

apply_text_formatting(title, bold=True, is_title=True)
apply_text_formatting(subtitle)

add_custom_logo(prs, slide)

# Save the presentation
prs.save("test_presentation.pptx")